<a href="https://colab.research.google.com/github/sollall/myidea/blob/sisaku/%E7%94%BB%E5%83%8F%E3%81%A7%E3%82%AE%E3%83%A3%E3%83%83%E3%83%97%E8%A8%88%E7%AE%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install condacolab
import condacolab
condacolab.install()

!mamba install -c conda-forge rdkit

from google.colab import output
output.clear()

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 1)

    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [62]:
from sklearn.model_selection import train_test_split
import pandas as pd

df=pd.read_csv("/content/drive/MyDrive/qm9.csv")
x=list(df["smiles"])
y=list(df["gap"])

x=np.asarray(x)
y=np.asarray(y)
y=(y-min(y))/(max(y)-min(y))

x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=0.8)

0.0


In [24]:
from rdkit import Chem
from rdkit.Chem import Draw

import numpy as np
from PIL import Image



def smiles2input(smiles):
    mol = Chem.MolFromSmiles(smiles)
    Draw.MolToFile(mol,"temp.png",size=(32,32))
    temp=np.array(Image.open("temp.png"))
    #Draw.MolToFile(mol,"test.png")

    temp=np.float32(temp)

    for i in range(len(temp)):
        for j in range(len(temp[i])):
            temp[i][j]=1-(temp[i][j]/255)
    temp/=3

    res=[[] for i in range(len(temp))]
    
    for i in range(len(temp)):
        for j in range(len(temp[i])):
            res[i].append(np.sum(temp[i][j],axis=0))
    
    res=np.array([[res]])
    res=torch.tensor(res)
    return res

smiles2input('CC(C)CC')

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])

In [25]:
torch.randn(1,1,300,300)

tensor([[[[ 2.4929,  0.8089, -0.4785,  ...,  0.5998, -0.9241,  1.3257],
          [-1.4561,  0.5671,  1.3573,  ...,  1.9645, -0.5374,  0.2269],
          [-2.4749, -0.8211, -0.4525,  ..., -0.1409,  1.2456, -0.4027],
          ...,
          [-0.2278, -0.7657,  0.4524,  ...,  0.4807,  0.2158,  0.2279],
          [ 2.0445, -0.3120,  3.1239,  ..., -1.6728, -1.2091,  0.7550],
          [-0.2529, -0.0704, -0.3758,  ...,  0.3289,  2.0847, -0.8232]]]])

In [27]:
net(smiles2input('CC(C)CC'))

tensor([[-0.0406,  0.0122,  0.0161,  0.0882, -0.0961, -0.0800, -0.0360, -0.0126,
          0.0343,  0.0574]], grad_fn=<AddmmBackward>)

In [26]:
net(torch.randn(1, 1, 32, 32))

tensor([[-0.0626,  0.0100,  0.0151,  0.0745, -0.0429, -0.0714, -0.0259, -0.0117,
          0.0153,  0.0288]], grad_fn=<AddmmBackward>)

In [30]:
target = torch.randn(10)  # a dummy target, for example
print(target)
target = target.view(1, -1)  # make it the same shape as output
print(target)

tensor([ 0.2994,  2.3114, -0.3772, -0.1755,  1.1682,  0.9018,  0.2545, -0.7810,
        -1.2781,  0.4275])
tensor([[ 0.2994,  2.3114, -0.3772, -0.1755,  1.1682,  0.9018,  0.2545, -0.7810,
         -1.2781,  0.4275]])


In [82]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
criterion = nn.MSELoss()

steps=5
loss=float("inf")
for step in range(steps):
    
    print(str(step+1)+"/"+str(steps))
    for index in range(len(x_train)):
        optimizer.zero_grad()
        output=net(smiles2input(x_train[index]))
        target=np.float(np.asarray(y_train[index]))
        target=torch.tensor(target)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if index%(len(x_train)//10)==0:
            print("　"+str(index)+"/"+str(len(x_train)))
            print("　"+str(loss))
    


    


1/5
　0/107108
　tensor(5.6495e-05, grad_fn=<MseLossBackward>)


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1, 10])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


　10710/107108
　tensor(0.0139, grad_fn=<MseLossBackward>)
　21420/107108
　tensor(0.0023, grad_fn=<MseLossBackward>)
　32130/107108
　tensor(0.0091, grad_fn=<MseLossBackward>)
　42840/107108
　tensor(0.0126, grad_fn=<MseLossBackward>)
　53550/107108
　tensor(0.0009, grad_fn=<MseLossBackward>)
　64260/107108
　tensor(0.0040, grad_fn=<MseLossBackward>)
　74970/107108
　tensor(0.0134, grad_fn=<MseLossBackward>)
　85680/107108
　tensor(0.0026, grad_fn=<MseLossBackward>)
　96390/107108
　tensor(0.0028, grad_fn=<MseLossBackward>)
　107100/107108
　tensor(0.0003, grad_fn=<MseLossBackward>)
2/5
　0/107108
　tensor(2.6015e-05, grad_fn=<MseLossBackward>)
　10710/107108
　tensor(0.0100, grad_fn=<MseLossBackward>)
　21420/107108
　tensor(0.0005, grad_fn=<MseLossBackward>)
　32130/107108
　tensor(0.0056, grad_fn=<MseLossBackward>)
　42840/107108
　tensor(0.0017, grad_fn=<MseLossBackward>)
　53550/107108
　tensor(0.0006, grad_fn=<MseLossBackward>)
　64260/107108
　tensor(0.0023, grad_fn=<MseLossBackward>)
　74970/107108
　tensor(0.00

In [63]:
torch.randn(1)

tensor([-0.0793])